In [1]:
#導入所需套件
import pandas as pd
import numpy as np
import os

path='Train_DataSet\\'
file_path=pd.read_excel("Train_Quality_DataSet\\quality_size.xlsx")['key']     #將file夾名稱放進file_path
right_list=list()
left_list=list()

#使用for迴圈設定完整的.xlsx路徑，讀進raw data後放入right_list及left_list
for i in file_path:
    dat_path=path+"file_"+str(i)
#     print(dat_path)
    excel_path=os.listdir(dat_path)
    for j in excel_path:
        if j=='pressure.xlsx':
            #print(dat_path+"\\"+j)
            A=pd.read_excel(dat_path+"\\"+j)
            A=A.drop(['Unnamed: 0','TimeStamp'], axis=1)
#               print(A.shape)                        
            for k in A:
                if k=='pressure_right':
                    right_list.append([A[k]])
                elif k=='pressure_left':
                    left_list.append([A[k]])
print("right_list_shape:",str(np.array(right_list).shape))
print("left_list_shape:",str(np.array(left_list).shape))

right_list_shape: (159, 1)
left_list_shape: (159, 1)


In [2]:
#導入所需套件
from keras.preprocessing import sequence

#使用pad_sequences進行data補值，並轉換list維度
cnt=0
for i in right_list:
    right_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("right_list_shape:",str(np.array(right_list).shape))                                  
right_list = np.reshape(right_list,(-1,7443))                           #三維轉二維
print("right_list_shape:",str(np.array(right_list).shape))              

cnt=0
for i in left_list:
    left_list[cnt]=sequence.pad_sequences(i,dtype='float64',maxlen=7443)          
    cnt+=1
print("left_list_shape:",str(np.array(left_list).shape))                                  
left_list = np.reshape(left_list,(-1,7443))                           #三維轉二維
print("left_list_shape:",str(np.array(left_list).shape))      

Using TensorFlow backend.


right_list_shape: (159, 1, 7443)
right_list_shape: (159, 7443)
left_list_shape: (159, 1, 7443)
left_list_shape: (159, 7443)


In [3]:
#合併right_list和left_list為feature
right_df=pd.DataFrame(right_list)
left_df=pd.DataFrame(left_list)
feature = pd.concat([right_df, left_df], axis=0)
feature.shape

(318, 7443)

In [4]:
#導入所需套件
from sklearn.utils import shuffle

labels=np.ones((318,),dtype='int64')
lbael_xlsx=pd.read_excel('Train_Quality_DataSet\\quality_size.xlsx')         #讀進quality_size.xlsx

#labels設定
for i in range(0,159):
    if lbael_xlsx['right'][i]>36.2:
        labels[i]=0
    
for i in range(159,318):
    if lbael_xlsx['left'][i-159]>36.2:
        labels[i]=0
        
#labels轉為DataFrame
labels=pd.DataFrame(labels)

In [5]:
#導入所需套件
from sklearn.model_selection import train_test_split,cross_val_score

#使用shuffle對feature、labels進行亂數配對
x,y=shuffle(feature,labels,random_state=2)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=2)

In [6]:
#導入所需套件
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

#使用LogisticRegression()訓練並預測
model=LogisticRegression(solver ='lbfgs',class_weight={0:0.1,1:0.9},max_iter=2000)
model=SVC(kernel ='sigmoid',gamma ='scale')
score=cross_val_score(model,x,y,cv=5)
model.fit(x,y)
print("Cross val score:",score)#[0.92307692 0.953125   0.9047619  0.93650794 0.95238095]
print("Average Cross val score:",score.mean())#0.9339705433455434

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y,

Cross val score: [0.92307692 0.9375     0.93650794 0.93650794 0.93650794]
Average Cross val score: 0.9340201465201465


In [7]:
#導入所需套件
from sklearn.externals import joblib

#模型儲存
joblib.dump(model,'gonogo_model.pkl')

D:\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['gonogo_model.pkl']